In [66]:
from airflow import DAG

from airflow.operators.python import PythonOperator


from time import strftime,localtime
from web_scraper import *

In [67]:
with DAG('player_box_scores_etl',default_args={'retries': 4},description='ETL DAG tutorial',
        schedule_interval='0 10 * 10 *',catchup=False,tags=['nba_stats']) as dag:

        dag.doc_md = __doc__

        def extract(**kwargs):
                ti = kwargs['ti']

                bs = Box_scores()
                year = int(strftime("%Y"))

                start_reg = 9
                end_reg = 4
                start_post = end_reg
                end_post = 7
                reg_season = True

                month = int(strftime('%m'))
                year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

                ti = kwargs['ti']
                if month <= end_reg or month >= start_reg:

                        url = bs.build_url(year_range, not reg_season)

                elif month >= start_post and month <= end_post:

                        url = bs.build_url(year_range, not reg_season)
                else:
                        return # exit dag
                
                return bs.iter_all(url,)

    


In [68]:
def get_last_year(year):
        if(year > 2000):
            return year % 2000
        return year % 100


In [69]:
import pendulum

In [70]:
d = pendulum.datetime(2022,5,27)

In [71]:
link = 'https://www.nba.com/stats/players/boxscores/?Season=2021-22&SeasonType=Regular%20Season&DateFrom=05%2F01%2F2021&DateTo=05%2F27%2F2022'

In [72]:
import datetime
from datetime import timedelta

d = datetime.datetime

e = d.strftime(datetime.date.today(),'%m-%d-%Y').replace('-','%2F')

In [73]:
s = link.find(e)
link[s:len(e)+s]

'05%2F27%2F2022'

In [74]:
e

'05%2F27%2F2022'

In [75]:
def extract(**kwargs):
# ti = kwargs['ti']
        d = datetime.datetime
        from web_scraper import Box_scores
        bs = Box_scores()
        year = int(d.strftime(datetime.date.today(),"%Y"))
        
        s = datetime.date.today() - timedelta(days= 13)

        start_reg = 9
        end_reg = 4
        start_post = end_reg
        end_post = 7
        reg_season = True

        month = int(strftime('%m'))
        year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

        # ti = kwargs['ti']
        if month <= end_reg or month >= start_reg:

                url = bs.build_url(year_range, not reg_season)

        elif month >= start_post and month <= end_post:

                url = bs.build_url(year_range,s, not reg_season)
        else:
                return # exit dag

        return bs.iter_all(url)


In [76]:
pages = []
for i in extract():
    pages.append(i)

In [77]:
def get_player_and_team_ids(self,html):
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find("table")
            
            pids = []
            tids = []
            gids = []
            
            player_reg = '/player/\d+'
            team_reg = '/team/\d+'
            game_reg = '/game/\d+'

            for l in table.find_all('a'):
                player_match = search(player_reg,l.get('href'))

                team_match = search(team_reg,l.get('href'))
                
                game_match = search(game_reg,l.get('href'))

                if(player_match):
                    player_id = int(search('\d+',player_match.group()).group())
                    pids.append(player_id)
                elif(team_match):
                    team_id = int(search('\d+',team_match.group()).group())
                    tids.append(team_id)
                elif(game_match):
                    game_id = search('\d+',game_match.group()).group()
                    gids.append(game_id)
                    
            return pids, tids, gids

In [78]:
import pandas as pd
from bs4 import BeautifulSoup

In [84]:
df = pd.DataFrame()
for i in pages:
    df = pd.concat((df,pd.read_html(i)[0]))

In [85]:
df.drop(columns=['Season','FP','FG%','FT%','3P%','REB'],inplace = True)

In [86]:
df.columns

Index(['Player', 'Team', 'Match Up', 'Game Date', 'W/L', 'MIN', 'PTS', 'FGM',
       'FGA', '3PM', '3PA', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', '+/-'],
      dtype='object')

In [87]:
di = {'3pm':'pm3', '3pa':'pa3','match\xa0up':'match_up','min':'mins',
'w/l': 'result','+/-':'plus_minus','game\xa0date':'game_date'}
# dicto = dict(df.columns,)
transform = [i.lower() for i in df.columns]
df.rename(str.lower,axis='columns',inplace=True)

df.rename(columns=di,inplace =True)

In [88]:
df.dtypes

player        object
team          object
match_up      object
game_date     object
result        object
mins           int64
pts            int64
fgm            int64
fga            int64
pm3            int64
pa3            int64
ftm            int64
fta            int64
oreb           int64
dreb           int64
ast            int64
stl            int64
blk            int64
tov            int64
pf             int64
plus_minus     int64
dtype: object

In [89]:
with open('shots.html','r') as f:
    h = f.read()

In [93]:
soup = BeautifulSoup(h)

In [95]:
soup.find('th')